# Tutorial: approaches to serverless code writing

In [7]:
import os
import warnings

warnings.filterwarnings('ignore')

In [8]:
from typing import List, Union, Optional, Dict

from qiskit import QuantumCircuit, transpile
from qiskit.providers import Backend
from qiskit.circuit.random import random_circuit
from qiskit.providers.aer import AerSimulator
from qiskit.providers.fake_provider import FakeVigo, FakeAlmaden, FakeBrooklyn, FakeCasablanca

from quantum_serverless import QuantumServerless, remote, get, put

---

There are multiple approaches to write serverless code using ray.

We will review 2 of them:
- exposing all details to user
- hiding details from user

In our examples we will be building functions and classes that can transpile circuits remotely and in parallel manner

### Approach #1: revealing details of implementations to user

One way of writing your modules, classes and functions in a way that user himself handling ray stuff.

Inside you module you will have something like that

In [9]:
# ==================
# Insude your module
# ==================

# your_module.transpiler

@remote
def remote_transpile(circuits: List[QuantumCircuit], backend: Backend):
    return transpile(circuits=circuits, backend=backend) 

Then user will use this funciton in following way

In [10]:
# ================
# User perspective
# ================

# from your_module.transpiler import remote_transpile

serverless = QuantumServerless()

with serverless.context():
    # First we need to put our object to cluster and pass object ids to functions

    # Let's do that for circuits
    circuits = [random_circuit(3, 2) for _ in range(10)]
    circuits_id = put(circuits)

    # and for backends
    backend_ids = []
    for fake_backend in [FakeAlmaden(), FakeBrooklyn(), FakeCasablanca(), FakeVigo()]:
        backend = AerSimulator.from_backend(fake_backend)
        backend_id = put(backend)
        backend_ids.append(backend_id)
    
    # now we need to call remote function and pass all objects to form tasks
    tasks = []
    for backend_id in backend_ids:
        task_id = remote_transpile.remote(circuits_id, backend_id)
        tasks.append(task_id)
        
    print(f"Results: {get(tasks)}")

Results: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87b852d810>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87b852cca0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87b852e0e0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87b852df90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87b852c040>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0d73ca0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0d71420>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0d72f50>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0d716c0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0e48fd0>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0e4a410>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c0e4b910>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87c881a3e0>, <qiskit.circuit.quantumcircuit.Quantum

---

As you can see user has to do a lot of builerplate code and pass ray objects and pointers across application.

### Approach #2: hiding details of implementations from user


We can do better by hiding all of this details inside our module. Let's have a look

In [11]:
# ==================
# Insude your module
# ==================

# your_module.transpiler

@remote
def remote_transpile(circuits: List[QuantumCircuit], backend: Backend):
    return transpile(circuits=circuits, backend=backend) 


def parallel_transpile(circuits: List[QuantumCircuit], backends: List[Backend]) -> List[QuantumCircuit]:
    circuits = [random_circuit(3, 2) for _ in range(10)]
    circuits_id = put(circuits)

    backend_ids = []
    for backend in backends:
        backend_id = put(backend)
        backend_ids.append(backend_id)
    
    # now we need to call remote function and pass all objects to form tasks
    tasks = []
    for backend_id in backend_ids:
        task_id = remote_transpile.remote(circuits_id, backend_id)
        tasks.append(task_id)
    
    return get(tasks)

Now let's see how code shrunks from users perspective, which is bringing better user experimce

In [12]:
# ================
# User perspective
# ================

# from your_module.transpiler import remote_transpile

circuits = [random_circuit(3, 2) for _ in range(10)]
backends = [AerSimulator.from_backend(backend)
            for backend in [FakeAlmaden(), FakeBrooklyn(), FakeCasablanca(), FakeVigo()]]

serverless = QuantumServerless()

with serverless.context():
    print(f"Results: {parallel_transpile(circuits, backends)}")

Results: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f8799b8d2a0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f8799b8df90>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f8799b8eef0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f8799b8d1e0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f8799b8dd20>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a09859c0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a09841c0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a0985ea0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a0987640>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a0987d00>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a8c8c8e0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a8c8ea40>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f87a8d05780>, <qiskit.circuit.quantumcircuit.Quantum

---

Now we have only 2 lines of code to call our serverless library.
It hides all unnecessary details from user.